In [1]:
from google.colab import drive
drive.mount('/content/drive')

#drive.flush_and_unmount()

!ls ./drive/MyDrive/ML/Projet2_Road_Segmentation/Ressources


%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import scipy.ndimage 
import pandas
import tensorflow as tf
import cv2
from PIL import Image
import time

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

root_dir = "./drive/MyDrive/ML/Projet2_Road_Segmentation/Ressources/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files)
imgs = [load_image(image_dir + files[i]) for i in range(n)]
gt_dir = root_dir + "groundtruth/"
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]




Mounted at /content/drive
groundtruth  test_set_images  training


In [4]:
#First of all we have to chose between offline expansion (expanding our whole dataset beforehand) and online augmentation (expanding mini-batches at each iteration)

#We know that online expansion is preferred for relatively large datasets because we can't necessarily handle the explosive increase in size of the dataset
#Our initial dataset is really small (only 100 elements). Therefore we can apply offline expansion in order to reach several thousands of inputs


#Show image
#cimg = concatenate_images(imgs_flipped[99], gt_imgs_flipped[99])
#fig1 = plt.figure(figsize=(10, 10))
#plt.imshow(cimg, cmap='Greys_r')

import cv2

def rotate_image(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0]) 
    abs_sin = abs(rotation_mat[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat

In [2]:
#1. Flip
def flip_images_lr(imgs,gt_imgs):
    n = len(imgs)
    imgs_flipped = [np.fliplr(imgs[i]) for i in range(n)]
    gt_imgs_flipped = [np.fliplr(gt_imgs[i]) for i in range(n)]
    return imgs_flipped, gt_imgs_flipped

def flip_images_ud(imgs,gt_imgs):
    n = len(imgs)
    imgs_flipped = [np.flipud(imgs[i]) for i in range(n)]
    gt_imgs_flipped = [np.flipud(gt_imgs[i]) for i in range(n)]
    return imgs_flipped, gt_imgs_flipped



def flip_pipeline(imgs,gt_imgs):
    imgs_flipped_lfr, gt_imgs_flipped_lfr = flip_images_lr(imgs,gt_imgs)
    imgs_flipped_ud, gt_imgs_flipped_ud = flip_images_ud(imgs,gt_imgs)
   
    return imgs_flipped_lfr + imgs_flipped_ud,gt_imgs_flipped_lfr+gt_imgs_flipped_ud
    
    
    
    

#flip_pipeline(imgs,gt_imgs) 


In [3]:
#2. Rotate

def rotate(imgs,gt_imgs,angle):
    n = len(imgs)
    imgs_rot = [scipy.ndimage.interpolation.rotate(imgs[i], angle, axes=(1, 0), reshape=False, output=None, order=3, mode='reflect', cval=0.0, prefilter=True) for i in range(n)]
    #imgs_rot_resized = [cv2.resize(imgs_rot[i], (400,400), interpolation = 0) for i in range(n)]
    gt_imgs_rot = [scipy.ndimage.interpolation.rotate(gt_imgs[i], angle, axes=(1, 0), reshape=False, output=None, order=3, mode='reflect', cval=0.0, prefilter=True) for i in range(n)]
    #gt_imgs_rot_resized = [cv2.resize(gt_imgs_rot[i], (400,400), interpolation = 0) for i in range(n)]
    return imgs_rot,gt_imgs_rot
    

    
def rotatation_pipeline(imgs,gt_imgs,nb_rotations):
    
    imgs_output = []
    gt_imgs_output = []
    stepSize = 360/nb_rotations
    angle = stepSize
    for i in range(nb_rotations):
        imgs_rot_tmp, gt_imgs_rot_tmp = rotate(imgs,gt_imgs,angle)
        imgs_output = imgs_output + imgs_rot_tmp
        gt_imgs_output = gt_imgs_output + gt_imgs_rot_tmp
        angle += stepSize
   
    return imgs_output,gt_imgs_output
    

#imgs_rotated, gt_imgs_rotated = rotatation_pipeline(imgs,gt_imgs,8)
#print(np.array(imgs_rotated).shape)    
#cimg = concatenate_images(imgs_rotated[0],gt_imgs_rotated[0])
#fig1 = plt.figure(figsize=(10, 10))
#plt.imshow(cimg, cmap='Greys_r')

In [4]:
#3. Crop

def random_crop_and_resize(image):
    width = image.shape[0]
    cropped_image = tf.image.random_crop(image, size = [125,125,3])
    print(type(image))
    cropped_image = tf.image.resize(cropped_image, [400,400])
    return cropped_image


def crop_and_resize_imgs(imgs,gt_imgs):
    n = len(imgs)
    
    gt_imgs = np.array(gt_imgs).reshape((-1,400,400,1)).tolist()
   
    cropped_imgs = []
    cropped_gt_imgs = []
    #we extract 5 differents crops for each image
    for i in range(100):
      image_crop = tf.image.crop_and_resize(imgs, [[0,0,0.5,0.5],[0.5,0,1,0.5],[0.25,0.25,0.75,0.75],[0,0.5,0.5,1],[0.5,0.5,1,1]], [i,i,i,i,i], [400,400], method='bilinear',extrapolation_value=0.0, name=None)
      gt_image_crop = tf.image.crop_and_resize(gt_imgs, [[0,0,0.5,0.5],[0.5,0,1,0.5],[0.25,0.25,0.75,0.75],[0,0.5,0.5,1],[0.5,0.5,1,1]], [i,i,i,i,i], [400,400], method='bilinear',extrapolation_value=0.0, name=None)

      cropped_imgs.append(image_crop)
      cropped_gt_imgs.append(gt_image_crop)

      print(i)
    
    #cropped_imgs = [tf.image.crop_and_resize(imgs, [[0,0,0.5,0.5],[0.5,0,1,0.5],[0.25,0.25,0.75,0.75],[0,0.5,0.5,1],[0.5,0.5,1,1]], [i,i,i,i,i], [400,400], method='bilinear',extrapolation_value=0.0, name=None) for i in range(100)]
    #cropped_gt_imgs = [tf.image.crop_and_resize(gt_imgs, [[0,0,0.5,0.5],[0.5,0,1,0.5],[0.25,0.25,0.75,0.75],[0,0.5,0.5,1],[0.5,0.5,1,1]], [i,i,i,i,i], [400,400], method='bilinear',extrapolation_value=0.0, name=None) for i in range(100)]

    #we have to adjuste the dimensions
    cropped_imgs = np.array(cropped_imgs).reshape((-1,400,400,3))
    cropped_gt_imgs = np.array(cropped_gt_imgs).reshape((-1,400,400))
    
    return cropped_imgs,cropped_gt_imgs

start = time.time()
#cropped,gt_cropped = crop_and_resize_imgs(imgs,gt_imgs)
end = time.time()

print("Time elapsed:", (end-start)/60, " minutes")

#print(cropped.shape)
#print(gt_cropped.shape)


#cropped = random_crop_and_resize(imgs[0])
#cimg = concatenate_images(cropped[4],gt_cropped[4])
#fig1 = plt.figure(figsize=(10, 10))
#plt.imshow(cimg, cmap='Greys_r')

Time elapsed: 2.7418136596679685e-07  minutes


In [8]:
def data_rotation_flip_pipeline(imgs,gt_imgs):
  # flip vertically and horizontally
  
  imgs_flipped, gt_imgs_flipped = flip_pipeline(imgs,gt_imgs)

  imgs_out = imgs + imgs_flipped
  gt_imgs_out = gt_imgs + gt_imgs_flipped

  print(np.array(imgs_out).shape)
  print(np.array(gt_imgs_out).shape)
  print("\n\n********************* flip step done *************************")

  #rotate by steps of 45 degress 
  
  imgs_rotated, gt_imgs_rotated = rotatation_pipeline(imgs_out,gt_imgs_out,8)

  imgs_out = imgs_rotated
  gt_imgs_out = gt_imgs_rotated

  print(np.array(imgs_out).shape)
  print(np.array(gt_imgs_out).shape)
  print("\n\n********************* rotation step done *************************")
  #extract 5 different crops for each image
  #cropped,gt_cropped = crop_and_resize_imgs(imgs_out,gt_imgs_out)

  #imgs_out = imgs_out + cropped
  #gt_imgs_out = gt_imgs_out + gt_cropped

  for i in range(len(imgs_out)):
    tmp_img = img_float_to_uint8(imgs_out[i])
    tmp_img_gt = img_float_to_uint8(gt_imgs_out[i])
    
    plt.imsave(os.path.join(root_dir + "/out/images/" ,str(i) + ".png"),tmp_img)
    plt.imsave(os.path.join(root_dir + "/out/ground_truth/" ,str(i) + ".png"),tmp_img_gt,cmap='Greys_r')

  print("\n\n********************* write to desk step done *************************")


  return imgs_out,gt_imgs_out


imgs_out,gt_imgs_out = data_rotation_flip_pipeline(imgs,gt_imgs)

(300, 400, 400, 3)
(300, 400, 400)


********************* flip step done *************************
(2400, 400, 400, 3)
(2400, 400, 400)


********************* rotation step done *************************


********************* write to desk step done *************************


In [28]:

drive.mount('/content/drive')

files = os.listdir(root_dir + "out/images/")
gt_files = os.listdir(root_dir + "out/ground_truth/")

def crop_all_images():
  for i in range(2400):
    print("\n\n****** i: ",i)
    image = load_image(root_dir + "out/images/" + str(i) + ".png")[:,:,:3]
    gt_image = load_image(root_dir + "out/ground_truth/" + str(i) + ".png")[:,:,0]
    
  
    #reshape images to make it work with tensorflow
    img = np.array(image).reshape((1,400,400,3)).tolist()
    gt_img = np.array(gt_image).reshape((1,400,400,1)).tolist()

    #extract five differenct crops from image
    image_crop = tf.image.crop_and_resize(img, [[0,0,0.5,0.5],[0.5,0,1,0.5],[0.25,0.25,0.75,0.75],[0,0.5,0.5,1],[0.5,0.5,1,1]], [0,0,0,0,0], [400,400], method='bilinear',extrapolation_value=0.0, name=None)
    gt_image_crop = tf.image.crop_and_resize(gt_img, [[0,0,0.5,0.5],[0.5,0,1,0.5],[0.25,0.25,0.75,0.75],[0,0.5,0.5,1],[0.5,0.5,1,1]], [0,0,0,0,0], [400,400], method='bilinear',extrapolation_value=0.0, name=None)

    #we have to adjust the dimensions to be able to save our images
    cropped_imgs = np.array(image_crop).reshape((-1,400,400,3))
    cropped_gt_imgs = np.array(gt_image_crop).reshape((-1,400,400))

    #we now want to write all of our crops to disk
    for j in range(len(cropped_imgs)):
      print("j: ",j)
      tmp_img = cropped_imgs[j]
      tmp_gt_img = cropped_gt_imgs[j]

      tmp_img = img_float_to_uint8(tmp_img)
      tmp_img_gt = img_float_to_uint8(tmp_gt_img)

      plt.imsave(os.path.join(root_dir + "/out/images/" , "cropped_" + str(i) + "_" + str(j)+ ".png"),tmp_img)
      plt.imsave(os.path.join(root_dir + "/out/ground_truth/" ,"cropped_" + str(i) + "_" + str(j)+ ".png"),tmp_img_gt,cmap='Greys_r')




crop_all_images()
#cimg = concatenate_images(imgs_out[2300],gt_imgs_out[2300])
#fig1 = plt.figure(figsize=(10, 10))
#plt.imshow(cimg, cmap='Greys_r')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


****** i:  0
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  2
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  3
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  4
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  5
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  6
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  7
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  8
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  9
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  10
j:  0
j:  1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide


Streaming output truncated to the last 5000 lines.


****** i:  1775
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1776
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1777
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1778
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1779
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1780
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1781
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1782
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1783
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1784
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1785
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1786
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1787
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1788
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1789
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1790
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1791
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1792
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1793
j:  0
j:  1
j:  2
j:  3
j:  4


****** i:  1794
j:  0
j:  1
j:  2
j

In [29]:
#write files to disk


#drive.mount('/content/drive')


#for i in range(len(imgs_out)):
#  tmp = img_float_to_uint8(gt_imgs_out[i])
#  plt.imsave(os.path.join(root_dir + "/out/ground_truth/" ,str(i) + ".png"),tmp,cmap='Greys_r')




drive.flush_and_unmount()


